In [1]:
%cd /content
!git clone -b main2 https://github.com/wadi-sudo/deepdrone.git

/content
Cloning into 'deepdrone'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 961 bytes | 961.00 KiB/s, done.


In [ ]:
!unzip deepdrone/data.zip

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Define the path to your dataset and the transforms to apply to each image
train_data_path = "/content/data-10classes/train"
test_data_path = "/content/data-10classes/test"
transform_train = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the train and test datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform_test)

# Define the batch size for training
batch_size = 32

# Define the data loaders for training and testing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Replace the last layer with a new layer that predicts the rotation angle
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Replace 4 with the number of rotation angles

# Define the loss function and optimizer for training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.arange(4).repeat(batch_size//4))  # Define the rotation angle labels
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1} train loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "/content/model.pth")


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Define the path to your dataset and the transforms to apply to each image
train_data_path = "/content/data-10classes/train"
test_data_path = "/content/data-10classes/test"
transform_train = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the train and test datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform_test)

# Define the batch size for training
batch_size = 32

# Define the data loaders for training and testing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the last layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Replace the last layer with a new layer that predicts the rotation angle
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Replace 4 with the number of rotation angles

# Define the loss function and optimizer for training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.arange(4).repeat(batch_size//4))  # Define the rotation angle labels
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1} train loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "/content/model.pth")


In [10]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Define the path to your dataset and the transforms to apply to each image
train_data_path = "/content/data-10classes/train"
test_data_path = "/content/data-10classes/test"
transform_train = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((256, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the train and test datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform_test)

# Define the batch size for training
batch_size = 32

# Define the data loaders for training and testing
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the last layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Replace the last layer with a new layer that predicts the rotation angle
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Replace 4 with the number of rotation angles

# Define the loss function and optimizer for training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Use the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1} train loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "/content/model.pth")


Epoch 1 train loss: 1.2811
Epoch 2 train loss: 0.8509
Epoch 3 train loss: 0.7506
Epoch 4 train loss: 0.6891
Epoch 5 train loss: 0.6507
Epoch 6 train loss: 0.6288
Epoch 7 train loss: 0.6100
Epoch 8 train loss: 0.5903
Epoch 9 train loss: 0.5748
Epoch 10 train loss: 0.5583
